## Week 2 Day 3

Now we get to more detail:

1. Different models

2. Structured Outputs

3. Guardrails

In [85]:
from dotenv import load_dotenv
from openai import AsyncOpenAI
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel, input_guardrail, output_guardrail, GuardrailFunctionOutput
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
from pydantic import BaseModel

In [26]:
load_dotenv(override=True)

True

In [27]:
openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

if deepseek_api_key:
    print(f"DeepSeek API Key exists and begins {deepseek_api_key[:3]}")
else:
    print("DeepSeek API Key not set (and this is optional)")

if groq_api_key:
    print(f"Groq API Key exists and begins {groq_api_key[:4]}")
else:
    print("Groq API Key not set (and this is optional)")

OpenAI API Key exists and begins sk-proj-
Google API Key exists and begins AI
DeepSeek API Key not set (and this is optional)
Groq API Key not set (and this is optional)


In [28]:
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

### It's easy to use any models with OpenAI compatible endpoints

In [29]:
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
DEEPSEEK_BASE_URL = "https://api.deepseek.com/v1"
GROQ_BASE_URL = "https://api.groq.com/openai/v1"

In [62]:

deepseek_client = AsyncOpenAI(base_url=DEEPSEEK_BASE_URL, api_key=deepseek_api_key)
gemini_client = AsyncOpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)
groq_client = AsyncOpenAI(base_url=GROQ_BASE_URL, api_key=groq_api_key)
ollama_client = AsyncOpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

llama_model = OpenAIChatCompletionsModel(model="llama3.1:8b", openai_client=ollama_client)
gemini_model = OpenAIChatCompletionsModel(model="gemini-2.5-flash", openai_client=gemini_client)
# llama3_3_model = OpenAIChatCompletionsModel(model="llama-3.3-70b-versatile", openai_client=groq_client)
ollama_model = OpenAIChatCompletionsModel(model="qwen3:4b", openai_client=ollama_client)

In [63]:
sales_agent1 = Agent(name="Llama Sales Agent", instructions=instructions1, model=llama_model)
sales_agent2 =  Agent(name="Gemini Sales Agent", instructions=instructions2, model=gemini_model)
sales_agent3  = Agent(name="Qwen3:4b Sales Agent",instructions=instructions3,model=ollama_model)

In [64]:
description = "Write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

In [65]:
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("chelakov@gmail.com")  # Change to your verified sender
    to_email = To("chelakov@gmail.com")  # Change to your recipient
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [66]:
subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model=gemini_model)
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold sales email")

html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model=gemini_model)
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body")

In [67]:
email_tools = [subject_tool, html_tool, send_html_email]

In [68]:
instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."


emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=email_tools,
    model=gemini_model,
    handoff_description="Convert an email to HTML and send it")

In [69]:
tools = [tool1, tool2, tool3]
handoffs = [emailer_agent]

In [70]:
sales_manager_instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
You can use the tools multiple times if you're not satisfied with the results from the first try.
 
3. Handoff for Sending: Pass ONLY the winning email draft to the 'Email Manager' agent. The Email Manager will take care of formatting and sending.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must hand off exactly ONE email to the Email Manager — never more than one.
"""


sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model=gemini_model)

message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Automated SDR"):
    result = await Runner.run(sales_manager, message)

## Check out the trace:

https://platform.openai.com/traces

In [59]:
class NameCheckOutput(BaseModel):
    is_name_in_message: bool
    name: str

guardrail_agent = Agent( 
    name="Name check",
    instructions="Check if the user is including someone's personal name in what they want you to do.",
    output_type=NameCheckOutput,
    model=gemini_model
)

In [60]:
@input_guardrail
async def guardrail_against_name(ctx, agent, message):
    result = await Runner.run(guardrail_agent, message, context=ctx.context)
    is_name_in_message = result.final_output.is_name_in_message
    return GuardrailFunctionOutput(output_info={"found_name": result.final_output},tripwire_triggered=is_name_in_message)

In [61]:
careful_sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=[emailer_agent],
    model=gemini_model,
    input_guardrails=[guardrail_against_name]
    )

message = "Send out a cold sales email addressed to Dear CEO from Head of Business Development"

with trace("Protected Automated SDR"):
    result = await Runner.run(careful_sales_manager, message)

## Check out the trace:

https://platform.openai.com/traces

In [ ]:

message = "Send out a cold sales email addressed to Dear CEO from Head of Business Development"

with trace("Protected Automated SDR"):
    result = await Runner.run(careful_sales_manager, message)

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercise</h2>
            <span style="color:#ff7800;">• Try different models<br/>• Add more input and output guardrails<br/>• Use structured outputs for the email generation
            </span>
        </td>
    </tr>
</table>

In [77]:
# 1. Guardrail to check for offensive or inappropriate content
class OffensiveContentOutput(BaseModel):
    is_offensive: bool
    reason: str

offensive_content_agent = Agent(
    name="Offensive Content Detector",
    instructions="""Check if the message contains offensive, inappropriate, or unprofessional language.
    This includes profanity, discriminatory language, threats, or harassment.
    Return true if ANY such content is detected.""",
    output_type=OffensiveContentOutput,
    model=gemini_model
)

@input_guardrail
async def guardrail_against_offensive_content(ctx, agent, message):
    result = await Runner.run(offensive_content_agent, message, context=ctx.context)
    is_offensive = result.final_output.is_offensive
    return GuardrailFunctionOutput(
        output_info={"offensive_check": result.final_output},
        tripwire_triggered=is_offensive
    )

In [78]:
# 2. Guardrail to check for competitor mentions
class CompetitorCheckOutput(BaseModel):
    mentions_competitor: bool
    competitor_name: str
    context: str

competitor_check_agent = Agent(
    name="Competitor Mention Detector",
    instructions="""Check if the message mentions any competitors in the compliance/audit software space.
    Common competitors include: Vanta, Drata, Secureframe, Laika, Strike Graph, TrustCloud.
    If a competitor is mentioned, note which one and in what context.""",
    output_type=CompetitorCheckOutput,
    model=gemini_model
)

@input_guardrail
async def guardrail_against_competitor_mentions(ctx, agent, message):
    result = await Runner.run(competitor_check_agent, message, context=ctx.context)
    mentions_competitor = result.final_output.mentions_competitor
    return GuardrailFunctionOutput(
        output_info={"competitor_check": result.final_output},
        tripwire_triggered=mentions_competitor
    )

In [79]:
# 3. Guardrail to check for spam patterns
class SpamCheckOutput(BaseModel):
    is_spam_like: bool
    spam_indicators: list[str]

spam_check_agent = Agent(
    name="Spam Pattern Detector",
    instructions="""Check if the request would likely result in spam-like content.
    Look for patterns like: excessive urgency, too many exclamation marks, 
    requests for money/wire transfers, suspicious links, overly aggressive CTAs,
    or requests to mass-send without proper targeting.""",
    output_type=SpamCheckOutput,
    model=gemini_model
)

@input_guardrail
async def guardrail_against_spam_patterns(ctx, agent, message):
    result = await Runner.run(spam_check_agent, message, context=ctx.context)
    is_spam = result.final_output.is_spam_like
    return GuardrailFunctionOutput(
        output_info={"spam_check": result.final_output},
        tripwire_triggered=is_spam
    )

In [81]:
# 4. Guardrail to validate request scope
class RequestScopeOutput(BaseModel):
    is_reasonable: bool
    issues: list[str]
    recipient_count_estimate: int

request_scope_agent = Agent(
    name="Request Scope Validator",
    instructions="""Check if the email request is reasonable in scope.
    Flag if:
    - Requesting to send to an unreasonably large number of recipients
    - Making impossible promises or claims
    - Requesting unethical actions
    - Scope is too vague or unclear""",
    output_type=RequestScopeOutput,
    model=gemini_model
)

@input_guardrail
async def guardrail_request_scope(ctx, agent, message):
    result = await Runner.run(request_scope_agent, message, context=ctx.context)
    is_unreasonable = not result.final_output.is_reasonable
    return GuardrailFunctionOutput(
        output_info={"scope_check": result.final_output},
        tripwire_triggered=is_unreasonable
    )

In [86]:
# ==========================================
# OUTPUT GUARDRAILS
# ==========================================

# 1. Guardrail to validate email quality
class EmailQualityOutput(BaseModel):
    is_high_quality: bool
    has_clear_value_prop: bool
    has_cta: bool
    is_appropriate_length: bool
    word_count: int
    issues: list[str]

email_quality_agent = Agent(
    name="Email Quality Validator",
    instructions="""Validate the quality of a cold sales email.
    Check that it:
    - Has a clear value proposition
    - Includes a call-to-action (CTA)
    - Is appropriate length (not too short <50 words, not too long >300 words)
    - Is professional and well-structured
    - Doesn't make unrealistic claims or guarantees
    - Has proper grammar and spelling
    List any issues found.""",
    output_type=EmailQualityOutput,
    model=gemini_model
)

@output_guardrail
async def validate_email_quality(ctx, agent, output):
    # Extract the email content from output
    email_content = str(output)
    result = await Runner.run(email_quality_agent, f"Validate this email:\n\n{email_content}", context=ctx.context)
    is_quality = result.final_output.is_high_quality
    return GuardrailFunctionOutput(
        output_info={"quality_check": result.final_output},
        tripwire_triggered=not is_quality
    )

In [87]:
# 2. Guardrail to check for data leakage in output
class DataLeakageOutput(BaseModel):
    contains_sensitive_data: bool
    data_types_found: list[str]
    severity: str  # "none", "low", "medium", "high"
    
data_leakage_agent = Agent(
    name="Data Leakage Detector",
    instructions="""Check if the email contains any sensitive or internal information that shouldn't be shared:
    - Internal employee names (beyond the sender's name)
    - Specific pricing details not meant for public disclosure
    - Customer/client names without permission
    - Internal URLs, system names, or infrastructure details
    - API keys, credentials, passwords, or other secrets
    - Confidential business metrics or data
    
    List what types of sensitive data were found if any, and rate the severity.""",
    output_type=DataLeakageOutput,
    model=gemini_model
)

@output_guardrail
async def validate_no_data_leakage(ctx, agent, output):
    email_content = str(output)
    result = await Runner.run(data_leakage_agent, f"Check this email for sensitive data:\n\n{email_content}", context=ctx.context)
    has_leakage = result.final_output.contains_sensitive_data
    return GuardrailFunctionOutput(
        output_info={"data_leakage_check": result.final_output},
        tripwire_triggered=has_leakage
    )

In [88]:
# 3. Guardrail to validate professional tone
class ToneCheckOutput(BaseModel):
    is_professional: bool
    tone_issues: list[str]
    tone_description: str
    formality_level: str  # "too_casual", "appropriate", "too_formal"

tone_check_agent = Agent(
    name="Professional Tone Validator",
    instructions="""Validate that the email maintains a professional, respectful tone.
    It should NOT be:
    - Too casual or unprofessional (excessive slang, emojis, etc.)
    - Overly aggressive or pushy
    - Condescending or arrogant
    - Too formal/robotic (should still be personable)
    
    Describe the tone, assess formality level, and list any issues.""",
    output_type=ToneCheckOutput,
    model=gemini_model
)

@output_guardrail
async def validate_professional_tone(ctx, agent, output):
    email_content = str(output)
    result = await Runner.run(tone_check_agent, f"Check the tone of this email:\n\n{email_content}", context=ctx.context)
    is_professional = result.final_output.is_professional
    return GuardrailFunctionOutput(
        output_info={"tone_check": result.final_output},
        tripwire_triggered=not is_professional
    )

In [ ]:
# 4. Guardrail to check for legal/compliance issues
class ComplianceCheckOutput(BaseModel):
    has_compliance_issues: bool
    issues_found: list[str]
    regulations_violated: list[str]

compliance_check_agent = Agent(
    name="Compliance Validator",
    instructions="""Check if the email has any legal or compliance issues:
    - Missing unsubscribe information for marketing emails
    - False or misleading claims
    - Violation of CAN-SPAM Act requirements
    - GDPR concerns if targeting EU recipients
    - Inappropriate use of recipient data
    - Making guarantees that could be considered false advertising
    
    List any issues and which regulations might be violated.""",
    output_type=ComplianceCheckOutput,
    model=gemini_model
)

@output_guardrail
async def validate_compliance(ctx, agent, output):
    email_content = str(output)
    result = await Runner.run(compliance_check_agent, f"Check this cold email for compliance issues:\n\n{email_content}", context=ctx.context)
    has_issues = result.final_output.has_compliance_issues
    return GuardrailFunctionOutput(
        output_info={"compliance_check": result.final_output},
        tripwire_triggered=has_issues
    )

In [90]:
# 5. Guardrail to detect hallucinations or false information
class FactCheckOutput(BaseModel):
    contains_false_info: bool
    unverifiable_claims: list[str]
    concerns: list[str]

fact_check_agent = Agent(
    name="Fact Checker",
    instructions="""Check if the email contains false or unverifiable information about ComplAI:
    - Claims about features that may not exist
    - Unrealistic statistics or metrics
    - False industry information
    - Exaggerated benefits
    - Unverifiable testimonials or case studies
    
    List any unverifiable claims or potential misinformation.""",
    output_type=FactCheckOutput,
    model=gemini_model
)

@output_guardrail
async def validate_factual_accuracy(ctx, agent, output):
    email_content = str(output)
    result = await Runner.run(fact_check_agent, f"Check this email for false or unverifiable information:\n\n{email_content}", context=ctx.context)
    contains_false_info = result.final_output.contains_false_info
    return GuardrailFunctionOutput(
        output_info={"fact_check": result.final_output},
        tripwire_triggered=contains_false_info
    )

In [93]:
# Create a fully protected sales manager with all guardrails
fully_protected_sales_manager = Agent(
    name="Fully Protected Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=[emailer_agent],
    model=gemini_model,
    input_guardrails=[
        guardrail_against_name,
        guardrail_against_offensive_content,
        guardrail_against_competitor_mentions,
        guardrail_against_spam_patterns,
        guardrail_request_scope
    ],
    output_guardrails=[
        validate_email_quality,
        validate_no_data_leakage,
        validate_professional_tone,
        validate_compliance,
        validate_factual_accuracy
    ]
)

# Test with various messages
message = "Send out a cold sales email addressed to Dear CEO from Head of Business Development"

with trace("Fully Protected Automated SDR"):
    result = await Runner.run(fully_protected_sales_manager, message)